In [231]:
#!/bin/python
#
# A Python wrapper for https://min-api.cryptocompare.com/
#
# Created on:       03/03/2018  Aditya Shirode
# Last modified:    03/03/2018  Aditya Shirode
#

In [188]:
import time
import yaml
import logging
import requests
import datetime
import pandas as pd

In [27]:
filename = 'api_queries.yaml'

In [145]:
with open(filename, 'r') as f:
    try:
        query_yaml = yaml.load(f)
    except yaml.YAMLError as e:
        logging.error(e)

In [59]:
API_ENDPOINT = query_yaml['api_endpoint']

In [115]:
def format_parameter(parameter):
    """ Format parameters for the query """
    if isinstance(parameter, list):
        return ','.join(parameter)
    else:
        return parameter

In [154]:
def get_url(query_name, **kwargs):
    """ Get formatted url for a required query 
        Pass the arguments to the query as keyword arguments
    """
    query_data = query_yaml[query_name]
    query_url = API_ENDPOINT + query_data['url']
    query_arguments = []
    # Check if all the required arguments are provided
    if 'required' in query_data['parameters'].keys() and \
    any(argument not in kwargs for argument in query_data['parameters']['required']):
        logging.error("Not all required arguments provided for {query}. "
                      "Required arguments are {args}.".format(query=query_name, args=query_data['parameters']['required']))
        return None
    else:
        possible_query_arguments = list(query_data.get('parameters', {}).get('required', {}).keys()) + \
        list(query_data.get('parameters', {}).get('additional',{}).keys())
        for argument, value in kwargs.items():
            if argument in possible_query_arguments:
                query_arguments.append("{argument}={value}".format(argument=argument, value=format_parameter(value)))
        
    query = query_url + '?' + '&'.join(query_arguments)
    return query

In [238]:
def query_cryptocompare(url):
    """ Query CryptoCompare API """
    try:
        response = requests.get(url).json()
    except Exception as e:
        logging.error("Failure while querying {query}. \n{err}".format(query=url, err=e))
        return None

    if not response or 'Response' not in response.keys():
        return response

    if response['Response'] is 'Error':
        logging.error("Failed to query {url}".format(url=url))
        return None
    return response

In [127]:
def convert_timestamp(timestamp):
    try:
        return datetime.datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    except Exception as e:
        logging.error(e)
        return None

In [129]:
def get_data(response):
    """ Separate query data from response """
    header = {key: (value if key != 'Data' else len(value)) for key, value in response.items()}
    data = response['Data']
    return header, data

In [171]:
def get_readable_df(response):
    """ Convert timestamp into readable datetime """
    header, data = get_data(response)
    try:
        df_data = pd.DataFrame(data)
        df_data = df_data.rename(columns={'time': 'timestamp'})
        df_data['time'] = df_data.timestamp.apply(convert_timestamp)
        df_data = df_data.set_index('time')
        del df_data['timestamp']
    except AttributeError as e:
        logging.error(e)
        return None
    return df_data

----------------------------------

In [162]:
# Defaults
CURR = 'USD'
EXCHANGE = 'CCCAGG'
COIN = 'BTC'
COIN_LIST = ['BTC', 'ETH']
EXCHANGES = ['Kucoin', 'Cryptopia', 'HitBTC']

In [158]:
def get_coin_list():
    """ Get coin list """
    resp = query_cryptocompare(get_url('coinlist'))
    header, data = get_data(resp)
    return data

In [185]:
coins = get_coin_list()
COIN_DB = pd.DataFrame.from_dict(coins, orient='index')
print(COIN_DB.head())

        CoinName PreMinedValue  Sponsored                   Url Algorithm  \
007     007 coin           N/A      False   /coins/007/overview    Scrypt   
1337        1337           N/A      False  /coins/1337/overview       X13   
1CR      1Credit           N/A      False   /coins/1cr/overview    Scrypt   
1ST   FirstBlood           N/A      False   /coins/1st/overview       N/A   
2015   2015 coin           N/A      False  /coins/2015/overview       X11   

     Symbol  Name SortOrder TotalCoinSupply TotalCoinsFreeFloat     Id  \
007     007   007       263          989800                 N/A   5294   
1337   1337  1337       577    314159265359                 N/A  20824   
1CR     1CR   1CR       374     92000000000                 N/A   5406   
1ST     1ST   1ST       759        93468691                 N/A  28328   
2015   2015  2015        33               0                 N/A   3744   

                    ImageUrl ProofType          FullName FullyPremined  \
007    /media/3505

In [241]:
def get_exchanges_list():
    """ Get a list of all exchanges on CryptoCompare """
    data = query_cryptocompare(get_url('exchanges'))
    return data

In [248]:
exchanges = get_exchanges_list()
EXCHANGE_DB = pd.DataFrame.from_dict(e, orient='index')
print(EXCHANGE_DB.head())

                                  BTC                             LTC  \
Abucoins              [PLN, EUR, USD]                           [BTC]   
BTC38                           [CNY]                      [BTC, CNY]   
BTCChina                   [USD, CNY]                      [CNY, BTC]   
BTCE        [RUB, CNH, GBP, USD, EUR]  [USD, GBP, CNH, BTC, RUB, EUR]   
BTCMarkets                      [AUD]                      [AUD, BTC]   

                                  ETH         XRP         ETC         BCH  \
Abucoins                   [PLN, BTC]       [BTC]       [BTC]  [BTC, PLN]   
BTC38                           [CNY]  [CNY, BTC]       [CNY]         NaN   
BTCChina                          NaN         NaN         NaN         NaN   
BTCE        [LTC, RUB, USD, BTC, EUR]         NaN         NaN         NaN   
BTCMarkets                 [AUD, BTC]  [BTC, AUD]  [BTC, AUD]  [AUD, BTC]   

            XST PINK   C2  CVC ...  RND*   HT  ELA TOPC WICC CHAT*  WPR  POA  \
Abucoins    NaN  N

In [252]:
EXCHANGE_DB['BTC']

Abucoins                                            [PLN, EUR, USD]
BTC38                                                         [CNY]
BTCChina                                                 [USD, CNY]
BTCE                                      [RUB, CNH, GBP, USD, EUR]
BTCMarkets                                                    [AUD]
BTCXIndia                                                       NaN
BTER                                                  [BITUSD, CNY]
BXinth                                                        [THB]
Binance                                                      [USDT]
Bit2C                                                         [ILS]
BitBay                                              [USD, EUR, PLN]
BitFlip                                        [RUB, UAH, USD, EUR]
BitGrail                                                        NaN
BitMarket                                                [EUR, PLN]
BitSquare         [BRL, PLN, ZAR, ILS, GBP, CNY,

In [166]:
def get_price(coin, to_curr=CURR, exchange=EXCHANGE, **kwargs):
    """ Get real time price of the coin """
    if isinstance(coin, list):
        return query_cryptocompare(
            get_url(
                'pricemulti',
                fsyms=coin,
                tsyms=to_curr,
                e=exchange,
                **kwargs
            )
        )
    else:
        return query_cryptocompare(
            get_url(
                'price',
                fsym=coin,
                tsyms=to_curr,
                e=exchange,
                **kwargs
            )
        )

In [186]:
print(get_price(COIN))
print(get_price(COIN_LIST))
print(get_price(COIN_LIST, ['USD', 'ETH', 'LTC']))

{'USD': 11209.35}
{'ETH': {'USD': 847.66}, 'BTC': {'USD': 11205.9}}
{'ETH': {'LTC': 4.1, 'USD': 847.66, 'ETH': 1}, 'BTC': {'LTC': 54.2, 'USD': 11205.9, 'ETH': 13.21}}


In [190]:
def get_historical_price_timestamp(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE, **kwargs):
    """ Get value of coin in currency at a particular timestamp """
    if isinstance(timestamp, datetime.datetime):
        timestamp = time.mktime(timestamp.timetuple())
    
    return query_cryptocompare(
        get_url(
            'pricehistorical',
            fsym=coin,
            tsyms=to_curr,
            ts=int(timestamp),
            e=exchange,
            **kwargs
        )
    )

In [191]:
print(get_historical_price_timestamp(COIN))

{'BTC': {'USD': 11466.52}}


In [216]:
def get_historical_price_day(coin, to_curr=CURR, timestamp=time.time(), exchange=EXCHANGE, allData='false', **kwargs):
    """ Get price per day for the past month """
    resp = query_cryptocompare(
        get_url(
            'histoday',
            fsym=coin,
            tsym=to_curr,
            e=exchange,
            allData=allData,
            **kwargs
        )
    )
    return get_readable_df(resp)

def get_historical_price_last_day(*args, **kwargs):
    """ Get price for last day """
    return get_historical_price_day(*args, **kwargs, limit=1)

In [197]:
print(get_historical_price_day(COIN))

                        close      high       low      open  volumefrom  \
time                                                                      
2018-02-01 16:00:00   8870.82   9147.93   7786.20   9114.73   322596.22   
2018-02-02 16:00:00   9251.27   9504.37   8194.68   8872.87   139226.07   
2018-02-03 16:00:00   8218.05   9400.99   7889.83   9251.27   164609.06   
2018-02-04 16:00:00   6937.08   8391.29   6627.31   8218.05   341828.54   
2018-02-05 16:00:00   7701.25   7932.38   5968.36   6936.43   495883.24   
2018-02-06 16:00:00   7592.72   8572.68   7208.86   7701.25   271450.37   
2018-02-07 16:00:00   8260.69   8643.94   7590.48   7593.78   193040.33   
2018-02-08 16:00:00   8696.83   8743.20   7775.36   8259.26   162279.68   
2018-02-09 16:00:00   8569.29   9081.49   8176.25   8696.83   155616.78   
2018-02-10 16:00:00   8084.61   8573.35   7862.31   8569.32   123293.84   
2018-02-11 16:00:00   8911.27   8997.34   8084.41   8084.61   124923.98   
2018-02-12 16:00:00   854

In [202]:
print(get_historical_price_last_day(coin='ZRX', to_curr='BTC', exchange='Binance'))

,close,high,low,open,volumefrom,volumeto
time,,,,,,
2018-03-02 16:00:00,0.00008,0.000086,0.000079,0.000085,1762322,144.73
2018-03-03 16:00:00,0.00008,0.000081,0.000079,0.000080,286341,22.81


In [206]:
def get_historical_price_day_full(*args, **kwargs):
    """ Get price per day for all time  """
    return get_historical_price_day(*args, **kwargs, allData='true')

In [207]:
print(get_historical_price_day_full(COIN))

                           close         high          low         open  \
time                                                                      
2010-07-16 17:00:00      0.04951      0.04951      0.04951      0.04951   
2010-07-17 17:00:00      0.08584      0.08585      0.05941      0.04951   
2010-07-18 17:00:00      0.08080      0.09307      0.07723      0.08584   
2010-07-19 17:00:00      0.07474      0.08181      0.07426      0.08080   
2010-07-20 17:00:00      0.07921      0.07921      0.06634      0.07474   
2010-07-21 17:00:00      0.05050      0.08181      0.05050      0.07921   
2010-07-22 17:00:00      0.06262      0.06767      0.05050      0.05050   
2010-07-23 17:00:00      0.05454      0.06161      0.05049      0.06262   
2010-07-24 17:00:00      0.05050      0.05941      0.05050      0.05454   
2010-07-25 17:00:00      0.05600      0.05600      0.05000      0.05050   
2010-07-26 17:00:00      0.06000      0.06050      0.05300      0.05600   
2010-07-27 17:00:00      

In [213]:
def get_historical_price_hour(coin, to_curr=CURR, exchange=EXCHANGE, limit=168, **kwargs):
    """ Get price per hour for past 7 days """
    resp = query_cryptocompare(
        get_url(
            'histohour',
            fsym=coin,
            tsym=to_curr,
            e=exchange,
            limit=limit,
            **kwargs
        )
    )
    
    return get_readable_df(resp)

def get_historical_price_last_hour(*args, **kwargs):
    """ Get price for the last hour """
    return get_historical_price_hour(*args, **kwargs, limit=1)

In [217]:
coin_hour = get_historical_price_hour(COIN)
print(coin_hour.head())

,close,high,low,open,volumefrom,volumeto
time,,,,,,
2018-02-24 20:00:00,9736.85,9773.75,9569.94,9604.29,2813.50,27427635.50
2018-02-24 21:00:00,9744.90,9761.85,9657.98,9734.85,2098.89,20507152.91
2018-02-24 22:00:00,9803.16,9873.79,9744.90,9744.90,4627.24,45559332.39
2018-02-24 23:00:00,9765.49,9869.19,9733.21,9803.16,2607.10,25683166.59
2018-02-25 00:00:00,9640.76,9807.69,9628.24,9765.49,3544.44,34513251.04


In [226]:
def get_historical_price_minute(coin, to_curr=CURR, exchange=EXCHANGE, toTs=time.time(), **kwargs):
    """ Get price per min for past 24 hours """
    resp = query_cryptocompare(
        get_url(
            'histominute',
            fsym=coin,
            tsym=to_curr,
            e=exchange,
            toTs=int(toTs),
            **kwargs
        )
    )
    return get_readable_df(resp)

In [227]:
print(get_historical_price_minute(COIN))

,close,high,low,open,volumefrom,volumeto
time,,,,,,
2018-02-24 20:00:00,9736.85,9773.75,9569.94,9604.29,2813.50,27427635.50
2018-02-24 21:00:00,9744.90,9761.85,9657.98,9734.85,2098.89,20507152.91
2018-02-24 22:00:00,9803.16,9873.79,9744.90,9744.90,4627.24,45559332.39
2018-02-24 23:00:00,9765.49,9869.19,9733.21,9803.16,2607.10,25683166.59
2018-02-25 00:00:00,9640.76,9807.69,9628.24,9765.49,3544.44,34513251.04
2018-02-25 01:00:00,9634.87,9680.51,9568.78,9641.44,3134.09,30269350.38
2018-02-25 02:00:00,9685.93,9730.42,9595.03,9634.72,2129.38,20650260.87
2018-02-25 03:00:00,9684.82,9762.53,9652.03,9685.93,2486.88,24246596.81
2018-02-25 04:00:00,9553.82,9721.31,9532.10,9684.82,3297.74,31775242.94


In [229]:
def get_historical_price_minute_by_day(*args, days_ago=0, **kwargs):
    """ Get price per min for 24 hours till days_ago """
    if days_ago > 7:
        logging.error("Can not get information by minute for more than 7 days. Getting information for last possible day.")
        days_ago = 7
    days_ago -= 1  # Subtracting one day as toTs considers ending timestamp
    ts = datetime.datetime.today() - datetime.timedelta(days_ago)
    ts = time.mktime(ts.timetuple())
    return get_historical_price_minute(*args, **kwargs, toTs=int(ts))

In [230]:
print(get_historical_price_minute_by_day(COIN, days_ago=7))

,close,high,low,open,volumefrom,volumeto
time,,,,,,
2018-02-18 20:00:00,10546.87,10599.84,10541.07,10555.30,1602.37,1.697759e+07
2018-02-18 21:00:00,10487.39,10580.04,10424.33,10547.46,3198.06,3.357902e+07
2018-02-18 22:00:00,10508.12,10536.18,10469.99,10490.08,1407.42,1.485090e+07
2018-02-18 23:00:00,10844.83,10874.74,10506.35,10508.15,6928.86,7.455671e+07
2018-02-19 00:00:00,10807.53,10860.22,10740.28,10846.15,4755.65,5.151725e+07
2018-02-19 01:00:00,10947.85,10961.28,10749.17,10807.53,4585.55,5.007194e+07
2018-02-19 02:00:00,10923.84,11061.68,10918.47,10947.85,7128.29,7.862320e+07
2018-02-19 03:00:00,10958.61,10989.08,10873.27,10924.30,3073.40,3.385431e+07
2018-02-19 04:00:00,10917.27,11026.95,10898.90,10958.61,3423.69,3.769516e+07
